In [2]:
# Wavefunction generation
from pyscf import mcscf, fci, lib 
from pyscf import gto, scf, tools, dft
from pyscf.dft import numint
import api as pyq 
import importlib
import os
import h5py
import pdb
import pyscf
import numpy as np
from pyscf.scf.hf import dump_scf_summary
print(pyq.__file__)
print(pyscf.__file__)

/Users/ksu/Documents/GitHub/pyqmc/pyqmc/api.py
/usr/local/anaconda3/lib/python3.9/site-packages/pyscf/__init__.py


In [5]:
mol = gto.M(atom="H 0. 0. 0.", spin=1, basis=f'ccecpccpvdz', unit='bohr')
mf = scf.UHF(mol)
dm = mf.init_guess_by_atom()
mf.kernel(dm)

converged SCF energy = -0.49972077380239  <S^2> = 0.75  2S+1 = 2


-0.4997207738023898

In [6]:
dm = mf.make_rdm1()
kin = mf.mol.intor_symmetric('int1e_kin')
ein = mf.mol.intor_symmetric('int1e_nuc')
j, k= mf.get_jk(mf.mol, dm)

In [14]:
#Ekin
ekin = np.einsum('ij,sji->', kin, dm).real
#EI
ei = np.einsum('ij,sji->', ein, dm).real
#EJ
ej = 0.5*np.einsum('sij,sji->', j, dm).real
ek = 0.5*np.einsum('sij,sji->', k, dm).real
#ENUC
enuc = mf.energy_nuc()
print('e1', ekin + ei, 'coul', ej, 'ek', ej, 'nuc', enuc)
#Compare to PYSCF
print(mf.scf_summary)

e1 -0.4997207738023897 coul 0.31241940086331144 ek 0.31241940086331144 nuc 0
{'e1': -0.4997207738023896, 'e2': -3.12815322383097e-17, 'nuc': 0}


In [13]:
mf.energy_tot()

-0.49972077380238966

In [16]:
a=10
delta = 0.2
xs = np.arange(-a, a, delta)
ys = np.arange(-a, a, delta)
zs = np.arange(-a, a, delta)
points = lib.cartesian_prod([xs, ys, zs])
ao_value = numint.eval_ao(mol, points)

In [18]:
ao_value.shape

(1000000, 5)